<a href="https://colab.research.google.com/github/hthomas229/PurpleCrown/blob/main/polars_tutorial_videos_1_thru_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#POLARS TUTORIAL

##Intro to Polars and Your First DataFrame

🐻‍❄️ Install and import polars


In [ ]:
!pip install polars

In [ ]:
import polars as pl
from rich import print #for pretty printing)

Create a polars series

In [ ]:
series = pl.Series("fruits", ["apple", "banana", "cherry"]) #column name and list of values
print(series)


Create a Polars Data Frame

In [ ]:
df = pl.DataFrame(  # a dictionary with column names as keys and values as lists
    {
        "name" :  ["Raj", "Bev", "Ken"],
        "age"  :  [11, 31, 42],
        "height": [4.5, 5.2, 6.3],
        "is_employed": [False, True, True]

    }
)
print(df)

🐻‍❄️ The first big difference between Polars and Pandas becomes obvious here:  NO INDEX.  This means no loc[] and no iloc[].

Polars provides a schema function for a quick glance


In [ ]:
print(df.schema)

Get a Glimpse of the Data

In [ ]:
df.glimpse()

Other common pandas functions like head and describe work also

In [ ]:
print(df.head(2))

In [ ]:
print(df.describe())

##Select Context Operations on DataFrames

🐻‍❄️ Note:  Polars is immutable. When we create result below, it returns a new dataframe.  There is no in place modification.

Select Specific Columns

In [ ]:
result = df.select("name", "age") # you can also pass a list of strings ["name", "age"]
print(result)

Rename Columns with Alias

In [ ]:
result = df.select(
    "name",
    pl.col("is_employed").alias("working?") # we use pl.col when we want to perform operations on a column

)
print(result)

Perform Basic Calculations

In [ ]:
#double the age
result = df.select(
    "name",
    pl.col("age") * 2
)
print(result)

Derived Column in Select

In [ ]:
result = df.select(
    "name",
    "height",
    (pl.col("height") > 6.0).alias("Tall")
)

print(result)

Create a Literal (Constant Column)

In [ ]:
result = df.select(
    pl.col("name"),
    pl.lit("name").alias("value")
)
print(result)

##Filtering Polars DataFrames


🐻‍❄️For the next few tutorials we will be using pokemon data. I will place the data file in my github repository https://github.com/hthomas229/PurpleCrown. Download it and upload to CoLab each time you run the notebook.

Scan Data from CSV - Lazy

🐻‍❄️ Lazy evaluation is one of the most powerful features of Polars.  We will cover it in detail in later tutorials.

In [ ]:
lf = pl.scan_csv("PokemonData.csv") #scans the csv contents
print(lf)

Read Data from CSV - Eager

🐻‍❄️ Since we are working with a small dataset we will load eager for the next few tutorials.



In [ ]:
df = pl.read_csv("PokemonData.csv") #reads the whole file into memory
print(df.head())

Basic Filter

In [ ]:
#Find fast pokemon
fast_pokemon = df.filter(pl.col("Speed") > 100).select("Name", "Speed")
print(fast_pokemon)

Multiple Conditions

`Polars uses ==, !=, &, | `

In [ ]:
result = df.filter(
    (pl.col("Attack") > 90) & (pl.col("Defense") > 80),

)
print(result)

String Filtering

In [ ]:
result = df.filter(
    pl.col("Type1") == "Fire"
    )

print(result)

Many Other String Methods



In [ ]:
result = df.filter(
    pl.col("Name").str.contains("saur")
)
print(result)

There will be a whole tutorial on string methods later.

Boolean Filtering

In [ ]:
#find pokemon who are Legendary or have a Speed higher than 130
result = df.filter(pl.col("Legendary") | (pl.col("Speed") >= 130)) # here "Legendary" only returns True values
print(result)

## **with_columns:** Adding Custom Columns to Polars DataFrames

🐻‍❄️with_columns creates a new dataframe that contains the columns from the original dataframe and the new columns according to its input expressions

In [ ]:
print(df.schema)

Add a custom column

In [ ]:
# add a column TotAtk which equals Attack + SpAtk
result = df.with_columns(
    (pl.col("Attack") + pl.col("SpAtk")).alias("TotAtk")
)
print(result)

In [ ]:
#select only Name and TotAtk -- chaining context -- we start to see pipelines
result = df.with_columns(
    (pl.col("Attack") + pl.col("SpAtk")).alias("TotAtk")
).select("Name", "TotAtk")
print(result)


Using a Variable & Adding Multiple Columns

In [ ]:
total_power = (pl.col("Attack") + pl.col("Speed")  + pl.col("Defense"))

result = df.with_columns(
    total_power = total_power,
    avg_total_power = total_power.mean(), #round for example in video
    max_total_power = total_power.max()
)

print(result)

Create a Conditioal Column with pl.when().then().otherwise()  -- Case statement

In [ ]:
#create a SpeedTier column

result = df.with_columns(
    pl.when(pl.col("Speed") >= 100).then(pl.lit("Fast"))
    .when(pl.col("Speed") >= 70).then(pl.lit("Medium Speed"))
    .otherwise(pl.lit("Slow")).alias("SpeedTier")
  )
print(result)

##Sorting and Limiting Polars DataFrames

In [ ]:
#sort pokemon by name
result =df.sort("Name") # ascending is the default argument for sort
print(result.head())

In [ ]:
#sort pokemon by Speed descending and return the top 5

result = df.sort("Speed",descending=True).limit(5)
print(result)

In [ ]:
#sort by HP descending and return the five weakest pokemon -- select only name and HP

result = df.sort("HP", descending=True).select("Name", "HP").tail(5)
print(result)

Multi Column Sort

In [ ]:
# sort by Type 1 ascending then Attack Descending -- select only Name, Type1 and Attack

result = df.sort(
    ["Type1", "Attack"],
    descending=[False, True]
).select("Name", "Type1", "Attack")

print(result)

Top and Bottom K

In [ ]:
#get 5 fastest pokemon
result = df.top_k(5, by="Speed")
print(result)

In [ ]:
#get 5 slowest pokemon
result = df.bottom_k(5, by="Speed")
print(result)

##Grouping & Aggregating Polars DataFrames

Here are some of the most commonly used aggregation functions: `sum(),mean(),median(), min(), max(), len()`



In [ ]:
# sum pokemon HP by Type1 and rename to TotHP
result = df.group_by("Type1").agg(
    pl.col("HP").sum().alias("TotHP")
)
print(result)

In [ ]:
# group by Type1 and get the Average Attack,Average Speed, Median Defense, Max HP and Min HP

result= (df.group_by("Type1")
.agg(
    pl.col("Attack").mean().alias("AvgAttack"),
    pl.col("Speed").mean().alias("AvgSpeed"),
    pl.col("Defense").median().alias("MedianDefense"),
    pl.col("HP").max().alias("MaxHP"),
    pl.col("HP").min().alias("MinHP"))
)

print(result)

In [ ]:
# get the count of each Type1 pokemon
result = df.group_by("Type1").agg(
    pl.len().alias("Count")
)

print(result)

In [ ]:
#group by speed in 10s and get max and min for each
result = df.group_by(
    (pl.col("HP") // 10 * 10).alias("SpeedTier")
).agg(
    pl.len(),
    pl.col("HP").max().alias("fastest"),
    pl.col("HP").min().alias("slowest")
).sort("SpeedTier", descending=True)
print(result)

#Lazy Evaluation

##The Real Power of Polars

We can scan and manipulate huge amounts of data that Pandas can't.

## 📊 Data Scale: pandas vs Polars

### 🐼 How much data can pandas handle?

#### Rule of thumb
- **pandas is memory-bound**
- If it doesn’t fit in RAM → ⚠️ you’re in trouble

#### Limits in practice
- ✅ Comfortable: **hundreds of MB**
- ⚖️ OK with tuning: **1–5 GB**
- 😬 Painful / risky: **10+ GB**
- 🛑 Hard stop: **data size ≈ available RAM**

#### Why pandas struggles at scale
- **Eager execution** (everything loads immediately)
- **Python objects + NumPy arrays** → high memory overhead
- **No native query optimizer**
- **Frequent data copies** (often invisible)

#### You *can* stretch pandas with:
- Chunking (`chunksize`)
- Careful dtype optimization
- `categorical` columns  

…but at the cost of **manual complexity and fragile pipelines**.

---

### ⚡ How much data can Polars handle?

#### Rule of thumb
- **Polars scales beyond RAM**

#### Limits in practice
- ✅ Comfortable: **many GB**
- 🚀 Common: **10s–100s of GB**
- ⏳ Realistic upper bound: **disk space + patience**

#### Why Polars scales better
- Written in **Rust**
- Uses **Apache Arrow** (columnar, cache-friendly)
- **Lazy execution + query optimizer**
- **Streaming execution**
- **Zero-copy operations** where possible
- **Multi-threaded by default**





In [ ]:
import polars as pl
from rich import print


# Ways to Create a LazyFrame


1.   From dictionary
2.   From DataFrame with .lazy()
3.   pl.scan_



In [ ]:
#from Python dictionary
lf = pl.LazyFrame(
    {
        "tool" : ["hammer", "saw", "wrench"],
        "qty" : [3, 4, 5]
    }
)
print(lf)

In [ ]:
# using .lazy()
lf = pl.DataFrame(
    {
        "tool" : ["hammer", "saw", "wrench"],
        "qty" : [3, 4, 5]
    }
).lazy()
print(lf)

In [ ]:
#pl_scan from file
lf = pl.scan_csv("orders.csv")
print(lf)

To display the result we use the collect() method

In [ ]:
lf = pl.LazyFrame(
    {
        "tool" : ["hammer", "saw", "wrench"],
        "qty": [3, 5, 4]

    }
)
print(lf)



In [ ]:
print(lf.collect())



| Optimization          | Explanation                                                                 | Runs  |
|-----------------------|-----------------------------------------------------------------------------|-------|
| Predicate Pushdown    | Applies filters as early as possible at the scan level.                     | 1x    |
| Projection Pushdown   | Selects only the required columns at the scan level.                        | 1x    |
| Slice Pushdown        | Loads only the required slice at scan time and avoids materializing slices  | 1x    |

### Why this matters
- **Less data read** → faster execution  
- **Lower memory usage** → better scalability  
- **Works best with lazy execution engines** (e.g., Polars, Spark)


Simple Pushdown Example

In [ ]:
#filter tool column for hammer
query = lf.filter(
    pl.col("tool") == "hammer"
)
print(query)

In [ ]:
#collect
print(query.collect())

In [ ]:
#group by tools and sum quantity
query = lf.group_by("tool").agg("qty").sum()
print(query)

In [ ]:
#explain query
query.explain(optimized=True)

Graph

In [ ]:
#graph query
print(query.show_graph())

Execution


In [ ]:
lf = pl.scan_csv("orders.csv")
print(lf)

In [ ]:
#display the first 5 rows
print(lf.collect().limit(5))

In [ ]:
#change product to all lowercase and filter to only laptops
query = (
    lf.with_columns(pl.col("product").str.to_lowercase())
    .filter(pl.col("product") == "laptop")
).collect()

print(query)

In [ ]:
#explain

In [ ]:
#graph
print(query.show_graph())



As mentioned above to execute the query we call collect()

In [ ]:
#collect
print(query.collect())

If we we have more data than will fit in RAM we can pass the engine = "streaming"  argument to collect

In [ ]:
#streaming large amounts of data
query = (
    lf.with_columns(pl.col("product").str.to_lowercase())
    .filter(pl.col("product") == "laptop")
).collect(engine="streaming")

print(query)

#Basic Data Types & Casting






In [ ]:
import polars as pl
from rich import print

Integers (Int64, UInt32)

Floats (Float64)

Strings (Utf8)

Boolean

Date / Datetime / Duration

Categorical

Pyhton implicilty infers data types.  You can declare them explicitly using a schema/

In [ ]:
import polars as pl
from datetime import date, datetime, timedelta
pl.Config.set_tbl_cols(20)

schema = {
    "int_signed": pl.Int64,
    "int_unsigned": pl.UInt32,
    "float_col": pl.Float64,
    "string_col": pl.Utf8,
    "bool_col": pl.Boolean,
    "date_col": pl.Date,
    "datetime_col": pl.Datetime,
    "duration_col": pl.Duration,
    "category_col": pl.Categorical,
}

In [ ]:
from datetime import date, datetime, timedelta

df = pl.DataFrame(
    {
        "int_signed": [1, -5, 10],
        "int_unsigned": [1, 5, 10],
        "float_col": [1.5, 2.0, 3.75],
        "string_col": ["apple", "banana", "cherry"],
        "bool_col": [True, False, True],
        "date_col": [
            date(2024, 1, 1),
            date(2024, 1, 2),
            date(2024, 1, 3),
        ],
        "datetime_col": [
            datetime(2024, 1, 1, 9, 0),
            datetime(2024, 1, 2, 10, 30),
            datetime(2024, 1, 3, 14, 45),
        ],
        "duration_col": [
            timedelta(days=1),
            timedelta(hours=2),
            timedelta(minutes=30),
        ],
        "category_col": ["A", "B", "A"],
    },
    schema=schema,
)

print(df)
print(df.dtypes)


Casting


In [ ]:
df = pl.DataFrame({ "a": ["1", "2", "3"], "b": ["2026-01-01", "2026-01-02", "2026-01-03"] })
print(df)

In [ ]:
result = df.with_columns([ pl.col("a").cast(pl.Int64), pl.col("b").cast(pl.Date) ])
print(result)

SAFE v. STRICT

Polars can either:

strict: fail on invalid values

non‑strict: turn invalid values into nulls

In [ ]:
df = pl.DataFrame(
    {
        "raw_numbers": ["10", "20", "oops", "30"]
    }
)

print(df)
print(df.dtypes)

In [ ]:
#default strict=true
result = df.with_columns(
    pl.col("raw_numbers").cast(pl.Int64)
)

In [ ]:
result = df.with_columns(
    pl.col("raw_numbers").cast(pl.Int64, strict=False)
)
print(result)

#String Methods & Parsing

In [ ]:
import polars as pl
from rich import print

df = pl.DataFrame(
    {
        "tools": ["hammer🔨", "saw", "wrench", "screwdriver"],

    }
)

print(df)

len_bytes & len_chars

In [ ]:
#get the length in bytes and characters of each tool
result = df.with_columns(
    pl.col("tools").str.len_bytes().alias("byte_count"),
    pl.col("tools").str.len_chars().alias("char_count")
)
print(result)

Pattern Matching

In [ ]:
#pattern matching
result = df.select(
    pl.col("tools"),
    pl.col("tools").str.starts_with("ham").alias("starts_with_ham"),
    pl.col("tools").str.contains("ench").alias("contain_ench"),
    pl.col("tools").str.ends_with("driver").alias("ends_with_driver")
)
print(result)

String Methods




In [ ]:
words = pl.DataFrame(
    {
        "word": ["GEorge", "LIKES", "his", "cHickEn", "SpicY!"]
    }
)
print(words)

In [ ]:
#cases and zfill
result = words.with_columns(
    pl.col("word").str.to_lowercase().alias("lower"),
    pl.col("word").str.to_uppercase().alias("upper"),
    pl.col("word").str.to_titlecase().alias("title"),
    pl.col("word").str.zfill(10).alias("z_fill")
)
print(result)

Explode

In [ ]:
#explode
test = pl.DataFrame(
    {
    "A": ["Spicy", "Chicken", "Antidentite"]
}
)
print(test)

In [ ]:
print(test.select(pl.col("A").str.explode()))

Padded

In [ ]:
#pad strings with extra characters
result = test.with_columns(
    pl.col("A").str.pad_start(11, "*").alias("paddded")
)
print(result)

Replace


In [ ]:
#replace -- turn Chicken into Chicago
result = test.with_columns(
    pl.col("A").str.replace("ken", "ago").alias("replaced")
)
print(result)

Reverse

In [ ]:
#reverse the strings in column A of test
result = test.with_columns(
    pl.col("A").str.reverse().alias("backwards")
)
print(result)

Slice

In [ ]:
#make a slice of the 4th and 5th letters in the string
print(test.select("A", substr=pl.col("A").str.slice(3, length=2)))


strip_prefix and strip_suffix

In [ ]:
#strip characters from the beginning of the string
print(test.select(pl.col("A").str.strip_prefix("Chi").alias("stripped_chi")))

In [ ]:
#strip characters from the end of the string
print(test.select(pl.col("A").str.strip_suffix("ite").alias("stripped_ite")))

Extract


In [ ]:
product_codes = pl.DataFrame(
    {
        "item_description": [
            "Product_A_ID:123",
            "Product_B_ID:456_v2",
            "Product_C_NoID",
            "Product_D_ID:789-X",
            "Product_E"
        ]
    }
)
print(product_codes)

In [ ]:
# Extract the numbers following 'ID:' using a regex pattern
# The regex r'ID:(\d+)' captures one or more digits after 'ID:'
result = product_codes.with_columns(
    pl.col("item_description").str.extract(r"ID:(\d+)").alias("extracted_id")
)
print(result)

Parsing Date and Time Strings

In [ ]:
clock = pl.DataFrame(
    {
        "time": ["10:00", "11:00", "12:00"]
    }
)
print(clock)

In [ ]:
#format time as time'
print(clock.with_columns(pl.col("time").str.strptime(pl.Time, '%H:%M').alias("formatted")))

In [ ]:
calendar = pl.DataFrame(
    {
        "date": ["2026-05-10", "2026-07-04", "2026-10-31"]
    }
)
print(calendar)

In [ ]:
#format date as date
print(calendar.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("formatted")))

#Dates and Times


In [ ]:
from datetime import datetime, date

df = pl.DataFrame({
    "datetime" : [datetime(2025, 1, 1,3,30,56), datetime(2024,12,25, 5, 28, 47)]
})

df = df.with_columns([
    pl.col("datetime").dt.year().alias("year"),
    pl.col("datetime").dt.month().alias("month"),
    pl.col("datetime").dt.day().alias("day"),
    pl.col("datetime").dt.hour().alias("hour"),
    pl.col("datetime").dt.minute().alias("minute"),
    pl.col("datetime").dt.second().alias("second")
])
df

In [ ]:
from datetime import datetime
import polars as pl

df = pl.DataFrame({
     "start" : [datetime(2024,12,25, 5, 28, 47)],
     "end":   [datetime(2025, 1, 1,3,30,56)],
 })

df = df.with_columns(
    (pl.col("end") - pl.col("start")).alias("duration")
)
df

In [ ]:
df = pl.DataFrame(
    {
        "date": [date(2022, 1, 1), date(2022, 1, 2)],
        "string": ["2022-01-01", "2022-01-02"],
    }
)

result = df.select(
    pl.col("date").dt.to_string("%Y-%m-%d"),
    pl.col("string").str.to_datetime("%Y-%m-%d"),
)
print(result)

In [ ]:
result = df.select(
    pl.col("date").cast(pl.Int64).alias("days_since_epoch"),
    pl.col("datetime").cast(pl.Int64).alias("us_since_epoch"),
    pl.col("time").cast(pl.Int64).alias("ns_since_midnight"),

)
print(result)

In [ ]:
df = pl.DataFrame(
    {
        "date": [date(2022, 1, 1), date(2022, 1, 2)],
        "string": ["2022-01-01", "2022-01-02"],
    }
)

result = df.select(
    pl.col("date").dt.to_string("%Y-%m-%d"),
    pl.col("string").str.to_datetime("%Y-%m-%d"),
)
print(result)

#Null Handling

## 📦 Topics Covered

- Counting NULL values per column and row  
- Filtering rows with NULLs  
- Dropping NULLs (`drop_nulls`)  
- Filling NULLs with constants  
- Forward and backward filling  
- Conditional filling with expressions  
- Interpolating missing values  
- NULL vs NaN differences in Polars  
- LazyFrame optimizations for NULL handling  

## ⚡ Why Polars for Missing Data?

- Expression-based NULL handling  
- Works seamlessly in LazyFrames  
- Predicate and projection pushdown  
- Faster and more memory-efficient than pandas  


In [ ]:
import polars as pl
import numpy as np
from rich import print

df = pl.DataFrame({
    "id": [1, 2, 3, 4, 5],
    "sales": [100.0, None, 300.0, None, 500.0],
    "region": ["US", "EU", None, "US", None],
    "growth": [0.10, np.nan, 0.30, None, 0.50],
})

print(df)


Counting Nulls

In [ ]:
#count null values per column
result = df.select(pl.all().null_count())
print(result)

In [ ]:
#with explicit labels
result = df.select(pl.all().null_count().name.suffix("_nulls"))
print(result)

In [ ]:
#per row
result = df.with_columns(
    pl.sum_horizontal(pl.all().is_null()).alias("nulls_per_row")
)
print(result)

Filtering Nulls

In [ ]:
#by row
result = df.filter(
    pl.any_horizontal(pl.all().is_null())
)
print(result)

In [ ]:
#by row not null
result = df.filter(
    pl.all_horizontal(pl.all().is_not_null())
)
print(result)

In [ ]:
#specific column
result = df.filter(
    pl.col("region").is_null()
)
print(result)

Dropping Nulls -- Be Careful! ⚠️

In [ ]:
#drop any row with any nulls
result = df.drop_nulls()
print(result)

In [ ]:
#drop only row with nulls in sales and growth
result = df.drop_nulls(subset=["sales", "growth"])
print(result)

In [ ]:
result = df.drop_nans()
print(result)

Filling Nulls


Be Careful ⚠️



> “Missing data isn’t just an inconvenience — it’s a decision point. Every NULL forces you to choose between distortion and omission.”




In [ ]:
#with a constant
result = df.with_columns(
    pl.col("sales").fill_null(0),
    pl.col("region").fill_null("Unknown")
)
print(result)

In [ ]:
#forward
result = df.with_columns(
    pl.col("sales").fill_null(strategy="forward")

    )
print(result)

In [ ]:
#backward
result = df.with_columns(
    pl.col("sales").fill_null(strategy="backward")

    )
print(result)

In [ ]:
#conditional
result = df.with_columns(
    pl.when(pl.col("sales").is_null())
    .then(pl.col("sales").mean())
    .otherwise(pl.col("sales"))
    .alias("sales_filled_mean")
)
print(result)

In [ ]:
#linear interpolation
result = df.with_columns(
    pl.col("sales").interpolate()
)
print(result)

More on NaNs

In [ ]:
#count NaNs
result = df.select(
    pl.col("growth").is_nan().sum().alias("nans")
)
print(result)

⭐ The best way to deal NaNs is to first convert all NaNs to nulls and then clean them with the null handling methods.

In [ ]:
result = df.with_columns(
    pl.col("growth").fill_nan(None)
)
print(result)

#Expression Expansion

## Expression Expansion in Polars

Expression expansion allows you to write a **single expression** that can automatically expand into **multiple expressions**, depending on the schema of the DataFrame or LazyFrame it is applied to.

This feature is not just syntactic sugar 🧁— it enables a powerful application of **DRY (Don’t Repeat Yourself)** principles in your code.

### Why This Matters

- One expression can target **multiple columns**
- The expression expands into a **list of expressions**
- The underlying computation is written **once and reused**
- Code becomes **shorter, clearer, and easier to maintain**

### Key Idea

Instead of writing the same logic repeatedly for many columns, you define it **once**, and Polars applies it across all matching columns automatically.


In [ ]:
import polars as pl
from rich import print
pl.Config.set_tbl_rows(50)

In [ ]:
df = pl.DataFrame( {
    "day" : ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday" ],
    "am_high" : [32, 34, 33, 31, 29],
    "am_low"  : [23, 25, 26, 22, 25],
    "pm_high"  : [37, 36, 40, 38, 35],
    "pm_low"  : [24, 23, 25, 20, 26],
    "wind_speed" : [11.1, 12.3 ,9.5, 15.2, 8.0]
})
print(df)

shape: (5, 6)
┌───────────┬─────────┬────────┬─────────┬────────┬────────────┐
│ day       ┆ am_high ┆ am_low ┆ pm_high ┆ pm_low ┆ wind_speed │
│ ---       ┆ ---     ┆ ---    ┆ ---     ┆ ---    ┆ ---        │
│ str       ┆ i64     ┆ i64    ┆ i64     ┆ i64    ┆ f64        │
╞═══════════╪═════════╪════════╪═════════╪════════╪════════════╡
│ Monday    ┆ 32      ┆ 23     ┆ 37      ┆ 24     ┆ 11.1       │
│ Tuesday   ┆ 34      ┆ 25     ┆ 36      ┆ 23     ┆ 12.3       │
│ Wednesday ┆ 33      ┆ 26     ┆ 40      ┆ 25     ┆ 9.5        │
│ Thursday  ┆ 31      ┆ 22     ┆ 38      ┆ 20     ┆ 15.2       │
│ Friday    ┆ 29      ┆ 25     ┆ 35      ┆ 26     ┆ 8.0        │
└───────────┴─────────┴────────┴─────────┴────────┴────────────┘

Explicit Expansion by Column Name

In [ ]:
#convert temp columns from fahrenheit to Celsius
result = df.with_columns(
    ((pl.col("am_high",
      "am_low",
      "pm_high",
      "pm_low") -32) * (5/9)).round(1)
)
print(result)


shape: (5, 6)
┌───────────┬─────────┬────────┬─────────┬────────┬────────────┐
│ day       ┆ am_high ┆ am_low ┆ pm_high ┆ pm_low ┆ wind_speed │
│ ---       ┆ ---     ┆ ---    ┆ ---     ┆ ---    ┆ ---        │
│ str       ┆ f64     ┆ f64    ┆ f64     ┆ f64    ┆ f64        │
╞═══════════╪═════════╪════════╪═════════╪════════╪════════════╡
│ Monday    ┆ 0.0     ┆ -5.0   ┆ 2.8     ┆ -4.4   ┆ 11.1       │
│ Tuesday   ┆ 1.1     ┆ -3.9   ┆ 2.2     ┆ -5.0   ┆ 12.3       │
│ Wednesday ┆ 0.6     ┆ -3.3   ┆ 4.4     ┆ -3.9   ┆ 9.5        │
│ Thursday  ┆ -0.6    ┆ -5.6   ┆ 3.3     ┆ -6.7   ┆ 15.2       │
│ Friday    ┆ -1.7    ┆ -3.9   ┆ 1.7     ┆ -3.3   ┆ 8.0        │
└───────────┴─────────┴────────┴─────────┴────────┴────────────┘

By DataType

In [ ]:
result = df.with_columns(
    ((pl.col(pl.Int64)-32) * (5/9)).round(1))
print(result)

shape: (5, 6)
┌───────────┬─────────┬────────┬─────────┬────────┬────────────┐
│ day       ┆ am_high ┆ am_low ┆ pm_high ┆ pm_low ┆ wind_speed │
│ ---       ┆ ---     ┆ ---    ┆ ---     ┆ ---    ┆ ---        │
│ str       ┆ f64     ┆ f64    ┆ f64     ┆ f64    ┆ f64        │
╞═══════════╪═════════╪════════╪═════════╪════════╪════════════╡
│ Monday    ┆ 0.0     ┆ -5.0   ┆ 2.8     ┆ -4.4   ┆ 11.1       │
│ Tuesday   ┆ 1.1     ┆ -3.9   ┆ 2.2     ┆ -5.0   ┆ 12.3       │
│ Wednesday ┆ 0.6     ┆ -3.3   ┆ 4.4     ┆ -3.9   ┆ 9.5        │
│ Thursday  ┆ -0.6    ┆ -5.6   ┆ 3.3     ┆ -6.7   ┆ 15.2       │
│ Friday    ┆ -1.7    ┆ -3.9   ┆ 1.7     ┆ -3.3   ┆ 8.0        │
└───────────┴─────────┴────────┴─────────┴────────┴────────────┘

Alll & Exclude

In [ ]:
result = df.select(pl.all())
print(result)

shape: (5, 6)
┌───────────┬─────────┬────────┬─────────┬────────┬────────────┐
│ day       ┆ am_high ┆ am_low ┆ pm_high ┆ pm_low ┆ wind_speed │
│ ---       ┆ ---     ┆ ---    ┆ ---     ┆ ---    ┆ ---        │
│ str       ┆ i64     ┆ i64    ┆ i64     ┆ i64    ┆ f64        │
╞═══════════╪═════════╪════════╪═════════╪════════╪════════════╡
│ Monday    ┆ 32      ┆ 23     ┆ 37      ┆ 24     ┆ 11.1       │
│ Tuesday   ┆ 34      ┆ 25     ┆ 36      ┆ 23     ┆ 12.3       │
│ Wednesday ┆ 33      ┆ 26     ┆ 40      ┆ 25     ┆ 9.5        │
│ Thursday  ┆ 31      ┆ 22     ┆ 38      ┆ 20     ┆ 15.2       │
│ Friday    ┆ 29      ┆ 25     ┆ 35      ┆ 26     ┆ 8.0        │
└───────────┴─────────┴────────┴─────────┴────────┴────────────┘

In [ ]:
result = df.select(pl.all().exclude("^am_.*$"))
print(result)

shape: (5, 4)
┌───────────┬─────────┬────────┬────────────┐
│ day       ┆ pm_high ┆ pm_low ┆ wind_speed │
│ ---       ┆ ---     ┆ ---    ┆ ---        │
│ str       ┆ i64     ┆ i64    ┆ f64        │
╞═══════════╪═════════╪════════╪════════════╡
│ Monday    ┆ 37      ┆ 24     ┆ 11.1       │
│ Tuesday   ┆ 36      ┆ 23     ┆ 12.3       │
│ Wednesday ┆ 40      ┆ 25     ┆ 9.5        │
│ Thursday  ┆ 38      ┆ 20     ┆ 15.2       │
│ Friday    ┆ 35      ┆ 26     ┆ 8.0        │
└───────────┴─────────┴────────┴────────────┘

Renaming the New Columns

In [ ]:
result_celsius = df.with_columns(
    ((pl.col("^(am)_(high|low)$") - 32) * (5/9)).name.suffix("_celsius")
)
print(result_celsius)

shape: (5, 8)
┌───────────┬─────────┬────────┬─────────┬────────┬────────────┬─────────────────┬────────────────┐
│ day       ┆ am_high ┆ am_low ┆ pm_high ┆ pm_low ┆ wind_speed ┆ am_high_celsius ┆ am_low_celsius │
│ ---       ┆ ---     ┆ ---    ┆ ---     ┆ ---    ┆ ---        ┆ ---             ┆ ---            │
│ str       ┆ i64     ┆ i64    ┆ i64     ┆ i64    ┆ f64        ┆ f64             ┆ f64            │
╞═══════════╪═════════╪════════╪═════════╪════════╪════════════╪═════════════════╪════════════════╡
│ Monday    ┆ 32      ┆ 23     ┆ 37      ┆ 24     ┆ 11.1       ┆ 0.0             ┆ -5.0           │
│ Tuesday   ┆ 34      ┆ 25     ┆ 36      ┆ 23     ┆ 12.3       ┆ 1.111111        ┆ -3.888889      │
│ Wednesday ┆ 33      ┆ 26     ┆ 40      ┆ 25     ┆ 9.5        ┆ 0.555556        ┆ -3.333333      │
│ Thursday  ┆ 31      ┆ 22     ┆ 38      ┆ 20     ┆ 15.2       ┆ -0.555556       ┆ -5.555556      │
│ Friday    ┆ 29      ┆ 25     ┆ 35      ┆ 26     ┆ 8.0        ┆ -1.666667       ┆ -3.888889      │
└───────────┴─────────┴────────┴─────────┴────────┴────────────┴─────────────────┴────────────────┘

Selectors

Polars comes with the submodule ***selectors*** that provides a number of functions that allow you to write more flexible column selections for expression expansion.

In [ ]:
import polars.selectors as cs

In [ ]:
result = df.select(cs.string() | cs.ends_with("_speed"))
print(result)

shape: (5, 2)
┌───────────┬────────────┐
│ day       ┆ wind_speed │
│ ---       ┆ ---        │
│ str       ┆ f64        │
╞═══════════╪════════════╡
│ Monday    ┆ 11.1       │
│ Tuesday   ┆ 12.3       │
│ Wednesday ┆ 9.5        │
│ Thursday  ┆ 15.2       │
│ Friday    ┆ 8.0        │
└───────────┴────────────┘

Combine Selectors with Expression Expansion

In [ ]:
result = df.select("day", (cs.contains("speed")- cs.string()) * 2)
print(result)

shape: (5, 2)
┌───────────┬────────────┐
│ day       ┆ wind_speed │
│ ---       ┆ ---        │
│ str       ┆ f64        │
╞═══════════╪════════════╡
│ Monday    ┆ 22.2       │
│ Tuesday   ┆ 24.6       │
│ Wednesday ┆ 19.0       │
│ Thursday  ┆ 30.4       │
│ Friday    ┆ 16.0       │
└───────────┴────────────┘

##Unique and Value Counts in Polars DataFrames





In [ ]:
df = pl.DataFrame({
    "color": ["red", "blue", "red", "green", "blue", "blue"]
})


In [ ]:
print(df)

shape: (6, 1)
┌───────┐
│ color │
│ ---   │
│ str   │
╞═══════╡
│ red   │
│ blue  │
│ red   │
│ green │
│ blue  │
│ blue  │
└───────┘

In [ ]:
# n_unique counts distinct values
print(df.select(pl.col("color")).n_unique())

3

In [ ]:
# unique returns the values themselves
print(df.select(pl.col("color")).unique())

shape: (3, 1)
┌───────┐
│ color │
│ ---   │
│ str   │
╞═══════╡
│ red   │
│ blue  │
│ green │
└───────┘

In [ ]:
# value_counts returns one row per value with counts

print(df.select(pl.col("color").value_counts()))

#notice this returns a struct -- a data type we will discuss later

shape: (3, 1)
┌─────────────┐
│ color       │
│ ---         │
│ struct[2]   │
╞═════════════╡
│ {"blue",3}  │
│ {"green",1} │
│ {"red",2}   │
└─────────────┘

In [ ]:
#approx_n_unique tries to appproximate in cases of very large dataframes
import numpy as np
large_df = pl.DataFrame({"nums":  np.random.randint(0, 100_000, 100_000)})


In [ ]:

result = large_df.select(
    pl.col("nums").n_unique().alias("n_unique"),
    pl.col("nums").approx_n_unique().alias("approx")
)

print(result)

shape: (1, 2)
┌──────────┬────────┐
│ n_unique ┆ approx │
│ ---      ┆ ---    │
│ u32      ┆ u32    │
╞══════════╪════════╡
│ 63376    ┆ 64083  │
└──────────┴────────┘